In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

# https://docs.sympy.org/latest/index.html

sp.init_printing() 

In [ ]:
#initialize symbols and time functions
t, x_0, y_0, v_0, Theta, g, m =sp.symbols(r't x_0 y_0 v_0 \Theta g m') 
g_vector=sp.Matrix([[0],[-g]])
x, y =sp.symbols(r'x y', cls=sp.Function)
x, y= x(t), y(t)
#x =sp.Function('x')(t)
#y =sp.Function('y')(t)

""" x_dot = sp.Derivative(x,t)
x_ddot = x_dot.diff(t)
y_dot = sp.Derivative(y,t)
y_ddot = y_dot.diff(t) """

position = sp.Matrix([[x],[y]])
velocity = position.diff(t)
acceleration = velocity.diff(t)
motion_eqt = sp.Eq(m*acceleration,m*g_vector)
motion_eqt


In [ ]:
ode = m*acceleration-m*g_vector
ode_flattened = ode.T.tolist()[0]
initial_conditions={x.subs(t,0): x_0, 
                    x.diff(t).subs(t, 0): v_0 * sp.cos(Theta),
                    y.subs(t,0): y_0, 
                    y.diff(t).subs(t, 0): v_0 * sp.sin(Theta),
                    }
x_sol_eq, y_sol_eq = sp.dsolve(ode_flattened,[x,y],ics=initial_conditions)
x_sol, y_sol = x_sol_eq.rhs, y_sol_eq.rhs

x_sol, y_sol, x_sol_eq, y_sol_eq

In [ ]:
# numeric functions
numeric_x_sol = sp.lambdify((t, x_0, y_0, v_0, Theta, g, m), x_sol)
numeric_y_sol = sp.lambdify((t, x_0, y_0, v_0, Theta, g, m), y_sol)
numeric_vx_sol = sp.lambdify((t, x_0, y_0, v_0, Theta, g, m), x_sol.diff(t))
numeric_vy_sol = sp.lambdify((t, x_0, y_0, v_0, Theta, g, m), y_sol.diff(t))


In [ ]:
#plot trajectory 
Nbr_points=100
time_line = np.linspace(0,1,Nbr_points)
num_x = numeric_x_sol(time_line, 0, 5, 8, np.pi/4, 10, 0.5)
num_y = numeric_y_sol(time_line, 0, 5, 8, np.pi/4, 10, 0.5)
# vector field
num_vx = numeric_vx_sol(time_line, 0, 5, 8, np.pi/4, 10, 0.5)
num_vy = numeric_vy_sol(time_line, 0, 5, 8, np.pi/4, 10, 0.5)

In [ ]:

fig_t, ax_t = plt.subplots(nrows=1, ncols=2, constrained_layout=True, figsize=(10, 4))
ax_t[0].plot(num_x,num_y)
ax_t[0].set_xlabel("x(m)")
ax_t[0].set_ylabel("y(m)")
ax_t[1].quiver(num_x,num_y,num_vx, num_vy, units='dots', scale=1, color='b')
plt.xlabel("x(m)")
ax_t[1].set_ylabel("y(m)")

fig, ax = plt.subplots(nrows=2, ncols=1, constrained_layout=True)
ax[0].plot(time_line, [num_vx]*Nbr_points)
ax[0].set_ylabel("v_x")
ax[1].plot(time_line,num_vy)
ax[1].set_ylabel("v_y")

# common axis labels
fig.supxlabel("t(s)")

In [ ]:
# y = y(x) 
x_x, y_y = sp.symbols('x_x y_y')
""" y_y = sp.symbols('y_y', cls=sp.Function)
y_y=y_y(x_x) """
x_as_a_symbol_eq = x_sol_eq.subs(x,x_x)
tx_sol = sp.solve(x_as_a_symbol_eq,t)[0]
y_as_a_symbol_eq = y_sol_eq.subs(y,y_y)
y_x_sol = y_as_a_symbol_eq.subs(t,tx_sol)
#y_x_sol=sp.trigsimp(y_x_sol)
sp.pprint(y_x_sol)

# find vertex
y_dot_prime = y_x_sol.rhs.diff(x_x)
x_heighest = sp.solve(sp.Eq(y_dot_prime,0),x_x)[0]
sp.pprint(x_heighest.diff(Theta))
Thata_highest=sp.solve(sp.Eq(x_heighest.diff(Theta),0),Theta)
print(Thata_highest)

# numeric
numeric_y_x = sp.lambdify((x_x, x_0, y_0, v_0, Theta, g, m), y_x_sol)
